In [1]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: outcome in c:\users\achin\anaconda3\lib\site-packages (from trio~=0.17->selenium) (1.2.0)



In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def get_credentials_from_file(filename):
#     Extracts email and password from the given file. Write the file in the same directory as follows:
#     bits mail
#     mail password (google)
#     smtp mail user name
#     smtp key
#     smtp email-id   
    with open(filename, 'r') as file:


        email = file.readline().strip()
        password = file.readline().strip()
    return email, password

browser = webdriver.Chrome()

browser.get('https://onboard.bits-pilani.ac.in/ucp.php?mode=login&redirect=index.php')

# Click on Google login button using the provided HTML info
google_login_btn = browser.find_element(By.XPATH, '//a[@title="BITS Email  Login"]')
google_login_btn.click()

# Retrieve credentials from file
email, password = get_credentials_from_file('sensitive.txt')

# Find email field and fill it in
email_field = browser.find_element(By.ID, 'identifierId')
email_field.send_keys(email)
next_btn = browser.find_element(By.ID, 'identifierNext')
next_btn.click()


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait until password field is available and visible, then select it
password_field = WebDriverWait(browser, 10).until(
    EC.visibility_of_element_located((By.NAME, 'Passwd'))
)


# Assuming a delay, fill in the password
password_field.send_keys(password)

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import time

MAX_RETRIES = 3
retry_count = 0

while retry_count < MAX_RETRIES:
    try:
        # Wait until the button is present and clickable.
        wait = WebDriverWait(browser, 10)
        signin_btn = wait.until(EC.element_to_be_clickable((By.ID, 'passwordNext')))
        
        signin_btn.click()
        break
    except StaleElementReferenceException:
        # In case of stale element, retry after a short delay
        time.sleep(1)
        retry_count += 1
        continue
    except TimeoutException:
        retry_count += 1
        continue

if retry_count == MAX_RETRIES:
    print("Failed to click the button after maximum retries.")


try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, 'span.username'))
    WebDriverWait(browser, 10).until(element_present)

    # Get the current URL after signing in.
    current_url = browser.current_url
    print("Current URL:", current_url)

except TimeoutException:
    print("Timed out waiting for the new page to load after sign-in.")


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

BASE_URL = f'{current_url}&recent_topics_start='
MAX_PAGES = 10  # Adjust according to your needs

def get_notices_from_page(browser, url):
    notices = []
    
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    topics = soup.select('ul.topiclist.topics.collapsible li')
    
    for topic in topics:
        topic_element = topic.select_one('.topictitle')
        topic_title = topic_element.text.strip()
        topic_link = topic_element['href'].strip()  # Extracting the href attribute for the link
        last_post = topic.select_one('.lastpost span').text.strip()
        views = topic.select_one('.views').text.strip()
        
        notice_info = {
            'Title': topic_title,
            'Link': topic_link,  # Adding the link to the notice info dictionary
            'Last Post': last_post,
            'Views': views
        }
        
        notices.append(notice_info)
    
    return notices


all_notices = []
for i in range(0, MAX_PAGES * 5, 5):
    url = BASE_URL + str(i)
    all_notices.extend(get_notices_from_page(browser, url))

# Printing all notices
print("Notices examples:")

for notice in all_notices[:5]:
    print("Title:", notice['Title'])
    print("Link:", notice['Link']) 
    print("Last Post:", notice['Last Post'])
    print("Views:", notice['Views'])
    print('---------------------------------------')



Current URL: https://onboard.bits-pilani.ac.in/index.php?sid=88f7e44f02181de25849c8653b7c4b6a
Notices examples:
Title: The SunPedal Ride" by Mr. Sushil Reddy, an IITB alumnus
Link: ./viewtopic.php?f=26&t=7178
Last Post: Last post by onboard



 Tue Sep 26, 2023 10:35 am
Views: 2 Views
---------------------------------------
Notices examples:
Title: G20 University Connect Event on 26th September 2023, New Delhi
Link: ./viewtopic.php?f=26&t=7175
Last Post: Last post by director



 Tue Sep 26, 2023 10:28 am
Views: 3 Views
---------------------------------------
Notices examples:
Title: The SunPedal Ride
Link: ./viewtopic.php?f=26&t=7157
Last Post: Last post by chiefwarden



 Mon Sep 25, 2023 4:45 pm
Views: 76 Views
---------------------------------------
Notices examples:
Title: The SunPedal Ride
Link: ./viewtopic.php?f=30&t=7160
Last Post: Last post by chiefwarden



 Mon Sep 25, 2023 4:45 pm
Views: 7 Views
---------------------------------------
Notices examples:
Title: Holiday on the

In [3]:
pip install filetype


Note: you may need to restart the kernel to use updated packages.


In [4]:
# from PIL import Image
# from io import BytesIO
# import os
# import requests
# from bs4 import BeautifulSoup
# import filetype

# BASE_URL = "https://onboard.bits-pilani.ac.in/"

# def guess_extension_from_content(content):
#     print(content)
#     kind = filetype.guess(content)
#     if kind is None:
#         return ''
#     return kind.extension

# def sanitize_filename(filename):
#     invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
#     for char in invalid_chars:
#         filename = filename.replace(char, '_')
#     return filename

# def scrape_topic_content(url):
#     full_url = BASE_URL + url.lstrip('./')
    
#     browser.get(full_url)
    
#     soup = BeautifulSoup(browser.page_source, 'html.parser')

#     heading = soup.select_one('.topic-title').text.strip()

#     author = soup.select_one('.username-coloured').text.strip()

#     post_content = soup.select_one('.postbody .content')
#     content_text = post_content.text.strip()

#     attachments = []

#     for attach_div in soup.select('.attachbox, .inline-attachment'):
#         for attach in attach_div.select('.file'):
#             img_tag = attach.select_one('img.postimage')
#             link = img_tag['src'] if img_tag else None
#             filename = attach.select_one('dd').text.split()[0]  # Assumes filename is the first word
#             filesize = ' '.join(attach.select_one('dd').text.split()[1:4])  # Assumes next three words are filesize details

#             attachment = {
#                 "filename": filename,
#                 "filesize": filesize,
#                 "link": link
#             }
#             attachments.append(attachment)
#     from selenium.webdriver.common.action_chains import ActionChains
#     from selenium.webdriver.common.by import By
#     from selenium.webdriver.common.keys import Keys
#     import time


#     download_dir = "downloaded_attachments"
#     os.makedirs(download_dir, exist_ok=True)
#     downloaded_files = []

#     for attach in attachments:
#         browser.get(BASE_URL + attach["link"].lstrip('./'))

#         time.sleep(2)

#         if len(browser.find_elements(By.TAG_NAME, 'img')) > 0:
#             image = browser.find_element(By.TAG_NAME, 'img')
#             action = ActionChains(browser)
#             action.context_click(image).send_keys(Keys.ARROW_DOWN).send_keys(Keys.RETURN).perform()

#         elif len(browser.find_elements(By.CLASS_NAME, 'downloadButton')) > 0:
#             download_button = browser.find_element(By.CLASS_NAME, 'downloadButton')
#             download_button.click()

#         time.sleep(5)

#         safe_filename = sanitize_filename(attach["filename"])
#         filepath = os.path.join(download_dir, safe_filename)
#         downloaded_files.append(filepath)

#     return {
#         'heading': heading,
#         'author': author,
#         'content': content_text,
#         'attachments': downloaded_files
#     }

In [71]:
from PIL import Image
from io import BytesIO
import os
import requests
from bs4 import BeautifulSoup
import filetype

BASE_URL = "https://onboard.bits-pilani.ac.in/"

def scrape_topic_content(url):
    full_url = BASE_URL + url.lstrip('./')
    
    browser.get(full_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    postbody_div = soup.select_one('.postbody')

    content_html = str(postbody_div)

    heading = soup.select_one('.topic-title').text.strip()

    author = soup.select_one('.username-coloured').text.strip()
    
    return   {'heading': heading,
        'author': author,
        'content': content_html}



In [87]:
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.by import By

def download_images(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    img_tags = soup.find_all('img')
    
    if not os.path.exists('downloaded_images'):
        os.makedirs('downloaded_images')
    
    saved_image_data = []

    for img_tag in img_tags:
        img_url = img_tag['src']
        if img_url.startswith("./"):
            img_url = base_url + img_url[1:]

        browser.get(img_url)
        
        img_element = browser.find_element(By.TAG_NAME, "img")
        img_content = img_element.screenshot_as_png
        
        width = img_element.size["width"]
        height = img_element.size["height"]
        
        img_id = img_url.split('=')[-1]
        img_filename = os.path.join('downloaded_images', f"image_{img_id}.png")
        
        with open(img_filename, 'wb') as img_file:
            img_file.write(img_content)
            
        saved_image_data.append({"path": img_filename, "width": width, "height": height})

    return saved_image_data



In [88]:
from bs4 import BeautifulSoup
import os

def replace_img_src_with_cid(html_content, image_data_list):
    soup = BeautifulSoup(html_content, 'html.parser')
    img_tags = soup.find_all('img')
    
    for img_tag, image_data in zip(img_tags, image_data_list):
        img_name = os.path.basename(image_data["path"])
        img_tag["src"] = "cid:{}".format(img_name)
        img_tag["width"] = str(image_data["width"])
        img_tag["height"] = str(image_data["height"])
        
    return str(soup)



In [89]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email(email_content, recipient_email, subject_prefix="Automated scraped email"):
    # Download images and replace image URLs with Content-ID references
    image_data_list = download_images(email_content['content'], BASE_URL)
    email_content['content'] = replace_img_src_with_cid(email_content['content'], image_data_list)
    
    with open("sensitive.txt", 'r') as file:
        lines = file.readlines()
        SMTP_SERVER = "smtp.gmail.com"
        SMTP_PORT = 465
        SMTP_USERNAME = lines[2].strip()
        SMTP_PASSWORD = lines[3].strip()
        FROM_EMAIL = lines[4].strip()

    msg = MIMEMultipart()
    msg['From'] = FROM_EMAIL
    msg['To'] = recipient_email
    msg['Subject'] = f"{subject_prefix} - {email_content['heading']}"
    
    body = MIMEText(email_content['content'], 'html', 'utf-8')
    msg.attach(body)

    for img_data in image_data_list:
        img_path = img_data["path"]
        with open(img_path, "rb") as img_file:
            img_name = os.path.basename(img_path)
            img_mime = MIMEBase('image', 'png')
            img_mime.set_payload(img_file.read())
            encoders.encode_base64(img_mime)
            img_mime.add_header('Content-Disposition', f'inline; filename={img_name}')
            img_mime.add_header('Content-ID', f'<{img_name}>')
            msg.attach(img_mime)

    with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
        server.login(FROM_EMAIL, SMTP_PASSWORD)
        server.sendmail(FROM_EMAIL, recipient_email, msg.as_string())

    print("Sent mail..\n")


In [90]:
for notice in all_notices:
    scraped_content = scrape_topic_content(notice['Link'])
    send_email(scraped_content, "f20211457@pilani.bits-pilani.ac.in")

Sent mail..

